So far we've learnt how to scrape the web, and how to make a request for information from an API. Some websites make APIs even easier. Check out [RapidAPI](https://rapidapi.com/) they take care of writing most of the code for you.

We will use the [AeroDataBox API](https://rapidapi.com/aedbx-aedbx/api/aerodatabox/), which can retrieve all sorts of information about flights and airports. We will show you how to retrieve information about the airports, and then it's up to you to apply this, along with what you've already learnt this week, to **produce a function, which retrieves tomorrows flight information for the major airports in the cities you web scraped**.

In [5]:
import pandas as pd
import requests

On the left hand side of the AeroDataBox API page, you'll see a list of options for information that you can retrieve:
> - Flights API
- Subsciption / PUSH API
- Airport API
- Aircraft API
- Healthcheck & Status API

1. We want to select `Airport API`

2. Then within Airport API we want to select `Search airports by location`

3. Now in the middle third you'll want to enter the `latitude` and `longitude` of any city to test... we chose Berlin: latitude 52.31 longitude 13.24. Next we changed the `radiusKM` to only 50km. And finally set `withFlightInfoOnly` to true, so it will only return airports which have flight data (scheduled or live) available.

4. On the right hand third of the screen you should see a block of code that looks pretty unfamiliar. This is because by default the code is probably set to *(Node.js) Axios*. However, we have the power to change this to familiar python. Select the dropdown box at the top of the code and select `python > requests`.

Now you can copy the code to your notebook and it should look a little something like the cell below:

In [7]:
import requests

url = "https://aerodatabox.p.rapidapi.com/airports/search/location/52.31/13.24/km/50/16"

querystring = {"withFlightInfoOnly":"true"}

headers = {
	"X-RapidAPI-Key": "YOUR_API_KEY",
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"message":"You are not subscribed to this API."}


Let's view the response as `.json()` instead of `.text` so that it's easier to read

In [8]:
response.json()

{'message': 'You are not subscribed to this API.'}

We can now turn this into a dataframe using `.json_normalize()`

In [204]:
pd.json_normalize(response.json()['items'])

,icao,iata,name,shortName,municipalityName,countryCode,timeZone,location.lat,location.lon
0,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE,Europe/Berlin,52.35139,13.493889
1,EDDT,TXL,Berlin -Tegel,-Tegel,Berlin,DE,Europe/Berlin,52.55970,13.287699


Let's now use this for the latitude and longitude of multiple cities

In [3]:
def get_airports(latitudes, longitudes):
  # API headers
  headers = {
      "X-RapidAPI-Key": "YOUR_API_KEY",
      "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
  }

  querystring = {"withFlightInfoOnly": "true"}

  # DataFrame to store results
  all_airports = []

  for lat, lon in zip(latitudes, longitudes):
    # Construct the URL with the latitude and longitude
    url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{lat}/{lon}/km/50/16"

    # Make the API request
    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
      data = response.json()
      airports = pd.json_normalize(data.get('items', []))
      all_airports.append(airports)

  return pd.concat(all_airports, ignore_index=True)

In [ ]:
# coordinates for Berlin, Paris, London
latitudes = [52.5200, 48.8567, 51.5072]
longitudes = [13.4050, 2.3522, -0.1275]

get_airports(latitudes, longitudes)

,icao,iata,name,shortName,municipalityName,countryCode,timeZone,location.lat,location.lon
0,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE,Europe/Berlin,52.35139,13.493889
1,LFPB,LBG,Paris -Le Bourget,-Le Bourget,Paris,FR,Europe/Paris,48.96940,2.441390
2,LFPO,ORY,Paris -Orly,-Orly,Paris,FR,Europe/Paris,48.72530,2.359440
3,LFPG,CDG,Paris Charles de Gaulle,Charles de Gaulle,Paris,FR,Europe/Paris,49.01280,2.549999
4,EGLC,LCY,London City,City,London,GB,Europe/London,51.50530,0.055277
5,EGLL,LHR,London Heathrow,Heathrow,London,GB,Europe/London,51.47060,-0.461941
6,EGKR,KRH,Redhill Aerodrome,Aerodrome,Redhill,GB,Europe/London,51.21360,-0.138611
7,EGKK,LGW,London Gatwick,Gatwick,London,GB,Europe/London,51.14810,-0.190277
8,EGGW,LTN,London Luton,Luton,London,GB,Europe/London,51.87470,-0.368333
9,EGSS,STN,London Stansted,Stansted,London,GB,Europe/London,51.88500,0.234999


###### **Challenge:** Arrivals information
Using what you have been shown above, plus the skills you've learnt in the last couple of days:
1. In `AeroDataBox API` use the `Flight API` > `FIDS/Schedules: Airport departures and arrivals (by time range)` section
2. Fill out the parameters in the middle third and then copy the `python: requests` code from the right hand third
3. Explore the data you get back. What would be useful in your DataFrame and what can be excluded? Remember Gans wants to know about when people are arriving in the city
4. Make a DataFrame from the information you see as important
5. Condense everything you did above into a function that can take a list of ICAO codes as an input, and as an output gives you a DataFrame with the information for *tomorrows arrivals*

In [196]:
# step 1: Import libraries
import requests   # to send HTTP requests to the API
import pandas as pd  # to organize data in a DataFrame
from datetime import datetime, timedelta  # to calculate tomorrow's date


In [94]:
url = f'https://aerodatabox.p.rapidapi.com/flights/airports/%7BcodeType%7D/LGA/2025-11-21T00:00/2025-11-21T23:59?withLeg=true&direction=Both&withCancelled=true&withCodeshared=true&withCargo=true&withPrivate=true&withLocation=false' 


In [103]:
import pandas as pd
from sqlalchemy import create_engine

# Connect to your local SQL database
connection_string = "sqlite:///your_database.db"  # or your actual DB connection
engine = create_engine(connection_string)




In [109]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///your_database_file.db")  # Use your actual database file path


In [148]:
import pandas as pd
from sqlalchemy import create_engine

# Sample data
cities_data = {
    "city_id": [1, 2, 3],
    "city_name": ["Berlin", "Humburg", "Munich"],
    "latitude": [52.5200, 48.8567, 51.5072],   # renamed to match previous functions
    "longitude": [13.4050, 2.3522, -0.1275]    # renamed to match previous functions
}

# Create DataFrame
cities_df = pd.DataFrame(cities_data)

# Create SQLite engine
engine = create_engine("sqlite:///my_database.db")  # adjust path if needed

# Push to SQLite
cities_df.to_sql("cities", engine, index=False, if_exists="replace")

# Verify
print(pd.read_sql("SELECT * FROM cities", engine))


   city_id city_name  latitude  longitude
0        1    Berlin   52.5200    13.4050
1        2   Humburg   48.8567     2.3522
2        3    Munich   51.5072    -0.1275


In [152]:
import pandas as pd
import requests

# Your RapidAPI headers
headers = {
    "X-RapidAPI-Key": "01dc31e656msh0432440d6e53c36p16d9c9jsn28532a555271",
    "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

def get_airports_per_city(latitudes, longitudes, radius_km=30):
    """
    Fetch airports for given latitudes and longitudes.
    
    Parameters:
    - latitudes: list of city latitudes
    - longitudes: list of city longitudes
    - radius_km: search radius in kilometers (default 50)
    
    Returns:
    - pandas DataFrame with airport information
    """
    all_airports = []

    for lat, lon in zip(latitudes, longitudes):
        url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{lat}/{lon}/km/{radius_km}/16"
        querystring = {"withFlightInfoOnly": "true"}

        response = requests.get(url, headers=headers, params=querystring)

        if response.status_code == 200:
            data = response.json()
            if 'items' in data and len(data['items']) > 0:
                airports = pd.json_normalize(data['items'])
                
                # Rename location columns for easier access
                if 'location.latitude' in airports.columns and 'location.longitude' in airports.columns:
                    airports = airports.rename(columns={
                        'location.latitude': 'latitude',
                        'location.longitude': 'longitude'
                    })
                
                # Keep only useful columns
                columns_to_keep = ['icao', 'iata', 'name', 'municipality', 'latitude', 'longitude']
                airports = airports[[col for col in columns_to_keep if col in airports.columns]]
                
                all_airports.append(airports)
        else:
            print(f"Error fetching airports for {lat},{lon}: {response.status_code}")

    if all_airports:
        return pd.concat(all_airports, ignore_index=True)
    else:
        print("No airports data returned.")
        return pd.DataFrame()  # Return empty DataFrame if no data

airports_df = get_airports_per_city(latitudes, longitudes)
print(airports_df)


   icao iata                     name
0  EDDT  TXL            Berlin -Tegel
1  EDDB  BER       Berlin Brandenburg
2  LFPB  LBG        Paris -Le Bourget
3  LFPO  ORY              Paris -Orly
4  LFPG  CDG  Paris Charles de Gaulle
5  EGLC  LCY              London City
6  EGLL  LHR          London Heathrow


In [ ]:
------- #solution from Instructor#-------

In [155]:
import pandas as pd
#pip install python-dotenv
from dotenv import load_dotenv
import os
load_dotenv()  # Load .env into environment
import requests

In [156]:
import requests

latitude = 52.31
longitude = 13.24

base_url_w_endpoint = "https://aerodatabox.p.rapidapi.com/airports/search/location"

params = {"lat":latitude,"lon":longitude,"radiusKm":"50","limit":"10","withFlightInfoOnly":"true"}

headers = {
	"X-RapidAPI-Key": 'from pytz import timezone
from datetime import datetime, timedelta

def tomorrows_flight_arrivals(icao_list):

    # api_key = userdata.get('aeroBoxDataApi')
    api_key = os.getenv('X-RapidAPI-Key')

    berlin_timezone = timezone('Europe/Berlin')
    today = datetime.now(berlin_timezone).date()
    tomorrow = (today + timedelta(days=1))

    list_for_arrivals_df = []

    for icao in icao_list:

        times = [["00:00","11:59"],["12:00","23:59"]]

        for time in times:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"

            querystring = {"direction":"Arrival","withCancelled":"false"}

            headers = {
                "X-RapidAPI-Key": api_key,
                "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
                }

            response = requests.request("GET", url, headers=headers, params=querystring)

            if response.status_code == 200:

                flights_resp = response.json()

                arrivals_df = pd.json_normalize(flights_resp["arrivals"])[["number", "airline.name", "movement.scheduledTime.local", "movement.terminal", "movement.airport.name", "movement.airport.icao"]]
                arrivals_df = arrivals_df.rename(columns={"number": "flight_number", "airline.name": "airline", "movement.scheduledTime.local": "arrival_time", "movement.terminal": "arrival_terminal", "movement.airport.name": "departure_city", "movement.airport.icao": "departure_airport_icao"})
                arrivals_df["arrival_airport_icao"] = icao
                arrivals_df["data_retrieved_on"] = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")
                arrivals_df = arrivals_df[["arrival_airport_icao", "flight_number", "airline", "arrival_time", "arrival_terminal", "departure_city", "departure_airport_icao", "data_retrieved_on"]]

                # fixing arrival_time
                arrivals_df["arrival_time"] = arrivals_df["arrival_time"].str.split("+").str[0]

                list_for_arrivals_df.append(arrivals_df)

                return pd.concat(list_for_arrivals_df, ignore_index=True)
            else: continue'
,
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.get(base_url_w_endpoint, headers=headers, params=params)
response.url

'https://aerodatabox.p.rapidapi.com/airports/search/location?lat=52.31&lon=13.24&radiusKm=50&limit=10&withFlightInfoOnly=true'

In [157]:
print(response.text)

{"searchBy":{"lat":52.31,"lon":13.24},"count":2,"items":[{"icao":"EDDB","iata":"BER","name":"Berlin Brandenburg","shortName":"Brandenburg","municipalityName":"Berlin","location":{"lat":52.35139,"lon":13.493889},"countryCode":"DE","timeZone":"Europe/Berlin"},{"icao":"EDDT","iata":"TXL","name":"Berlin -Tegel","shortName":"-Tegel","municipalityName":"Berlin","location":{"lat":52.5597,"lon":13.287699},"countryCode":"DE","timeZone":"Europe/Berlin"}]}


In [158]:
#We can now turn this into a dataframe using `.json_normalize()`
pd.json_normalize(response.json()['items'])

,icao,iata,name,shortName,municipalityName,countryCode,timeZone,location.lat,location.lon
0,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE,Europe/Berlin,52.35139,13.493889
1,EDDT,TXL,Berlin -Tegel,-Tegel,Berlin,DE,Europe/Berlin,52.55970,13.287699


In [161]:
#Let's now use this for the latitude and longitude of multiple cities
def get_airports(latitudes, longitudes):

    base_url_w_endpoint = "https://aerodatabox.p.rapidapi.com/airports/search/location"
    headers = {"X-RapidAPI-Key": '01dc31e656msh0432440d6e53c36p16d9c9jsn28532a555271',
               "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"}

    # DataFrame to store results
    all_airports = []

    for lat, lon in zip(latitudes, longitudes):

        base_url_w_endpoint = "https://aerodatabox.p.rapidapi.com/airports/search/location"
        params = {"lat":lat,"lon":lon,"radiusKm":"50","limit":"10","withFlightInfoOnly":"true"}

        # Make the API request
        response = requests.get(base_url_w_endpoint, headers=headers, params=params)

        if response.status_code == 200:
            data = response.json()
            airports = pd.json_normalize(data.get('items', []))
            all_airports.append(airports)

    return pd.concat(all_airports, ignore_index=True)

In [167]:
def create_connection_string():
  schema = "etl_cities"
  host = "localhost"
  user = "root"
  password = surekha_1509
  port = 3306
  return f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [168]:
def fetch_cities_data(connection_string):
  return pd.read_sql("cities", con=connection_string)

In [169]:
connection_string = create_connection_string()
cities_df = fetch_cities_data(connection_string)
cities_df


NameError: name 'surekha_1509' is not defined

In [170]:
# coordinates for Berlin, Hamburg, Cologne
latitudes = cities_df.latitude.values.tolist()
latitudes

longitudes = cities_df.longitude.values.tolist()
longitudes

[13.405, 2.3522, -0.1275]

In [171]:
# coordinates for Berlin, Paris, London
# latitudes = [52.5200, 48.8567, 51.5072]
# longitudes = [13.4050, 2.3522, -0.1275]

airports_df = get_airports(latitudes, longitudes)
airports_df

,icao,iata,name,shortName,municipalityName,countryCode,timeZone,location.lat,location.lon
0,EDDT,TXL,Berlin -Tegel,-Tegel,Berlin,DE,Europe/Berlin,52.55970,13.287699
1,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE,Europe/Berlin,52.35139,13.493889
2,LFPB,LBG,Paris -Le Bourget,-Le Bourget,Paris,FR,Europe/Paris,48.96940,2.441390
3,LFPO,ORY,Paris -Orly,-Orly,Paris,FR,Europe/Paris,48.72530,2.359440
4,LFPG,CDG,Paris Charles de Gaulle,Charles de Gaulle,Paris,FR,Europe/Paris,49.01280,2.549999
5,EGLC,LCY,London City,City,London,GB,Europe/London,51.50530,0.055277
6,EGLL,LHR,London Heathrow,Heathrow,London,GB,Europe/London,51.47060,-0.461941
7,EGKR,KRH,Redhill Aerodrome,Aerodrome,Redhill,GB,Europe/London,51.21360,-0.138611
8,EGKK,LGW,London Gatwick,Gatwick,London,GB,Europe/London,51.14810,-0.190277
9,EGGW,LTN,London Luton,Luton,London,GB,Europe/London,51.87470,-0.368333


In [207]:
# push it to sql

airports_df.to_sql('airports',
                    if_exists='append',
                    con=connection_string,
                    index=False)

33

In [199]:
# read it from sql to check

airports_df = pd.read_sql("airports", con=connection_string)
airports_df

,icao,iata,name,shortName,municipalityName,countryCode,timeZone,location.lat,location.lon
0,EDDT,TXL,Berlin -Tegel,-Tegel,Berlin,DE,Europe/Berlin,52.55970,13.287699
1,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE,Europe/Berlin,52.35139,13.493889
2,LFPB,LBG,Paris -Le Bourget,-Le Bourget,Paris,FR,Europe/Paris,48.96940,2.441390
3,LFPO,ORY,Paris -Orly,-Orly,Paris,FR,Europe/Paris,48.72530,2.359440
4,LFPG,CDG,Paris Charles de Gaulle,Charles de Gaulle,Paris,FR,Europe/Paris,49.01280,2.549999
5,EGLC,LCY,London City,City,London,GB,Europe/London,51.50530,0.055277
6,EGLL,LHR,London Heathrow,Heathrow,London,GB,Europe/London,51.47060,-0.461941
7,EGKR,KRH,Redhill Aerodrome,Aerodrome,Redhill,GB,Europe/London,51.21360,-0.138611
8,EGKK,LGW,London Gatwick,Gatwick,London,GB,Europe/London,51.14810,-0.190277
9,EGGW,LTN,London Luton,Luton,London,GB,Europe/London,51.87470,-0.368333


In [ ]:
# **Challenge:** Arrivals information
Using what you have been shown above, plus the skills you've learnt in the last couple of days:
1. In `AeroDataBox API` use the `Flight API` > `FIDS/Schedules: Airport departures and arrivals (by time range)` section
2. Fill out the parameters in the middle third and then copy the `python: requests` code from the right hand third
3. Explore the data you get back. What would be useful in your DataFrame and what can be excluded? Remember Gans wants to know about when people are arriving in the city
4. Make a DataFrame from the information you see as important
5. Condense everything you did above into a function that can take a list of ICAO codes as an input, and as an output gives you a DataFrame with the information for *tomorrows arrivals*

In [179]:
from pytz import timezone
from datetime import datetime, timedelta

def tomorrows_flight_arrivals(icao_list):

    # api_key = userdata.get('aeroBoxDataApi')
    api_key = '01dc31e656msh0432440d6e53c36p16d9c9jsn28532a555271'

    berlin_timezone = timezone('Europe/Berlin')
    today = datetime.now(berlin_timezone).date()
    tomorrow = (today + timedelta(days=1))

    list_for_arrivals_df = []

    for icao in icao_list:

        times = [["00:00","11:59"],["12:00","23:59"]]

        for time in times:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"

            querystring = {"direction":"Arrival","withCancelled":"false"}

            headers = {
                "X-RapidAPI-Key":'01dc31e656msh0432440d6e53c36p16d9c9jsn28532a555271',
                "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
                }

            response = requests.request("GET", url, headers=headers, params=querystring)

            if response.status_code == 200:

                flights_resp = response.json()

                arrivals_df = pd.json_normalize(flights_resp["arrivals"])[["number", "airline.name", "movement.scheduledTime.local", "movement.terminal", "movement.airport.name", "movement.airport.icao"]]
                arrivals_df = arrivals_df.rename(columns={"number": "flight_number", "airline.name": "airline", "movement.scheduledTime.local": "arrival_time", "movement.terminal": "arrival_terminal", "movement.airport.name": "departure_city", "movement.airport.icao": "departure_airport_icao"})
                arrivals_df["arrival_airport_icao"] = icao
                arrivals_df["data_retrieved_on"] = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")
                arrivals_df = arrivals_df[["arrival_airport_icao", "flight_number", "airline", "arrival_time", "arrival_terminal", "departure_city", "departure_airport_icao", "data_retrieved_on"]]

                # fixing arrival_time
                arrivals_df["arrival_time"] = arrivals_df["arrival_time"].str.split("+").str[0]

                list_for_arrivals_df.append(arrivals_df)

                return pd.concat(list_for_arrivals_df, ignore_index=True)
            else: continue

In [180]:
icao_list = airports_df.icao.to_list()
icao_list

['EDDT',
 'EDDB',
 'LFPB',
 'LFPO',
 'LFPG',
 'EGLC',
 'EGLL',
 'EGKR',
 'EGKK',
 'EGGW',
 'EGSS']

In [181]:
tomorrows_flight_arrivals_df = tomorrows_flight_arrivals(icao_list)
tomorrows_flight_arrivals_df

,arrival_airport_icao,flight_number,airline,arrival_time,arrival_terminal,departure_city,departure_airport_icao,data_retrieved_on
0,EDDB,SM 2960,Air Cairo,2025-11-21 06:25,1,Hurghada,HEGN,2025-11-20 16:51:58
1,EDDB,HU 489,Hainan,2025-11-21 06:40,1,Beijing,ZBAA,2025-11-20 16:51:58
2,EDDB,QR 79,Qatar,2025-11-21 06:50,1,Doha,OTHH,2025-11-20 16:51:58
3,EDDB,FR 137,Ryanair,2025-11-21 07:30,2,Bologna,LIPE,2025-11-20 16:51:58
4,EDDB,DS 1185,Easyjet Switzerland,2025-11-21 07:30,1,Bâle/Mulhouse,LFSB,2025-11-20 16:51:58
...,...,...,...,...,...,...,...,...
111,EDDB,A3 1428,Aegean,2025-11-21 11:50,1,Frankfurt-am-Main,EDDF,2025-11-20 16:51:58
112,EDDB,AI 8869,Air India,2025-11-21 11:50,1,Frankfurt-am-Main,EDDF,2025-11-20 16:51:58
113,EDDB,UA 8998,United,2025-11-21 11:50,1,Frankfurt-am-Main,EDDF,2025-11-20 16:51:58
114,EDDB,U2 5318,easyJet,2025-11-21 11:50,1,Manchester,EGCC,2025-11-20 16:51:58


In [188]:
# read it from sql to check

tomorrows_flight_arrivals_df = pd.read_sql("flights", con=connection_string)
tomorrows_flight_arrivals_df

,arrival_airport_icao,flight_number,airline,arrival_time,arrival_terminal,departure_city,departure_airport_icao,data_retrieved_on
0,EDDB,SM 2960,Air Cairo,2025-11-21 06:25,1,Hurghada,HEGN,2025-11-20 16:51:58
1,EDDB,HU 489,Hainan,2025-11-21 06:40,1,Beijing,ZBAA,2025-11-20 16:51:58
2,EDDB,QR 79,Qatar,2025-11-21 06:50,1,Doha,OTHH,2025-11-20 16:51:58
3,EDDB,FR 137,Ryanair,2025-11-21 07:30,2,Bologna,LIPE,2025-11-20 16:51:58
4,EDDB,DS 1185,Easyjet Switzerland,2025-11-21 07:30,1,Bâle/Mulhouse,LFSB,2025-11-20 16:51:58
...,...,...,...,...,...,...,...,...
111,EDDB,A3 1428,Aegean,2025-11-21 11:50,1,Frankfurt-am-Main,EDDF,2025-11-20 16:51:58
112,EDDB,AI 8869,Air India,2025-11-21 11:50,1,Frankfurt-am-Main,EDDF,2025-11-20 16:51:58
113,EDDB,UA 8998,United,2025-11-21 11:50,1,Frankfurt-am-Main,EDDF,2025-11-20 16:51:58
114,EDDB,U2 5318,easyJet,2025-11-21 11:50,1,Manchester,EGCC,2025-11-20 16:51:58


In [191]:
# read it from sql to check

tomorrows_flight_arrivals_df = pd.read_sql("flights", con=connection_string)
tomorrows_flight_arrivals_df

,arrival_airport_icao,flight_number,airline,arrival_time,arrival_terminal,departure_city,departure_airport_icao,data_retrieved_on
0,EDDB,SM 2960,Air Cairo,2025-11-21 06:25,1,Hurghada,HEGN,2025-11-20 16:51:58
1,EDDB,HU 489,Hainan,2025-11-21 06:40,1,Beijing,ZBAA,2025-11-20 16:51:58
2,EDDB,QR 79,Qatar,2025-11-21 06:50,1,Doha,OTHH,2025-11-20 16:51:58
3,EDDB,FR 137,Ryanair,2025-11-21 07:30,2,Bologna,LIPE,2025-11-20 16:51:58
4,EDDB,DS 1185,Easyjet Switzerland,2025-11-21 07:30,1,Bâle/Mulhouse,LFSB,2025-11-20 16:51:58
...,...,...,...,...,...,...,...,...
111,EDDB,A3 1428,Aegean,2025-11-21 11:50,1,Frankfurt-am-Main,EDDF,2025-11-20 16:51:58
112,EDDB,AI 8869,Air India,2025-11-21 11:50,1,Frankfurt-am-Main,EDDF,2025-11-20 16:51:58
113,EDDB,UA 8998,United,2025-11-21 11:50,1,Frankfurt-am-Main,EDDF,2025-11-20 16:51:58
114,EDDB,U2 5318,easyJet,2025-11-21 11:50,1,Manchester,EGCC,2025-11-20 16:51:58


In [210]:
# push it to sql

tomorrows_flight_arrivals_df.to_sql('flights',
                    if_exists='replace',
                    con=connection_string,
                    index=False)

116

In [203]:
# read it from sql to check

tomorrows_flight_arrivals_df = pd.read_sql("flights", con=connection_string)
tomorrows_flight_arrivals_df

,arrival_airport_icao,flight_number,airline,arrival_time,arrival_terminal,departure_city,departure_airport_icao,data_retrieved_on
0,EDDB,SM 2960,Air Cairo,2025-11-21 06:25,1,Hurghada,HEGN,2025-11-20 16:51:58
1,EDDB,HU 489,Hainan,2025-11-21 06:40,1,Beijing,ZBAA,2025-11-20 16:51:58
2,EDDB,QR 79,Qatar,2025-11-21 06:50,1,Doha,OTHH,2025-11-20 16:51:58
3,EDDB,FR 137,Ryanair,2025-11-21 07:30,2,Bologna,LIPE,2025-11-20 16:51:58
4,EDDB,DS 1185,Easyjet Switzerland,2025-11-21 07:30,1,Bâle/Mulhouse,LFSB,2025-11-20 16:51:58
...,...,...,...,...,...,...,...,...
111,EDDB,A3 1428,Aegean,2025-11-21 11:50,1,Frankfurt-am-Main,EDDF,2025-11-20 16:51:58
112,EDDB,AI 8869,Air India,2025-11-21 11:50,1,Frankfurt-am-Main,EDDF,2025-11-20 16:51:58
113,EDDB,UA 8998,United,2025-11-21 11:50,1,Frankfurt-am-Main,EDDF,2025-11-20 16:51:58
114,EDDB,U2 5318,easyJet,2025-11-21 11:50,1,Manchester,EGCC,2025-11-20 16:51:58
